1. DATA LOADING

In [1]:
# NLP Libraries
import nltk
import re

# Graphic libraries


#import csv files


2. DATA CLEANING

3. EXPLORATIVE DATA ANALYSIS

5. FEATURE EXTRACTION (NLP PIPELINE)

6. MODEL DEFINITION (SVM, TRANSFORMER, CREARE IL MODELLO PER LA CLASSIFICAZIONE DEI POST SESSISTI USANDO TRAINING SET E TEST SET)

7. EVALUATION AND METRICS (VALUTARE IL MODELLO USANDO LE METRICHE DI PRECISION E RECALL)